In [3]:
%run ./__init__edwh__notebooks.ipynb

Python 3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:20:46) 
[GCC 9.4.0]
 * postgresql://postgres@pgpool:5432/backend
Done.


In [105]:
!pip install attrs nltk

In [140]:
from nltk.stem.snowball import SnowballStemmer 
import string, re
from nltk import download
download('stopwords')
stemmer=SnowballStemmer("dutch", ignore_stopwords=True)
stemmer.stem("hebben")
split_re = re.compile(f'[{string.punctuation}{string.whitespace}]+')
common = 'de het een die nu dat wat hij zij we hun hen ze al'.split(' ')

def stemmed(body):
    tokens = [t for t in split_re.split(body) if t]
    tokens = [t.lower() for t in tokens]
    tokens = [t for t in tokens if t not in common]
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)
print(stemmed('Een super lange tekst, die nu gestemd moet worden. Kijken hoe dat werkt, en of dat werkt!'))

super lang tekst gestemd moet worden kijk hoe werkt en of werkt


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [134]:
import ghost
ghost = reload(ghost)
SITE_CONFIG = {
    'url': 'https://archief.meteddie.nl/', 
    'adminAPIKey': os.getenv('GHOST_ADMIN_API_KEY'),
    'contentAPIKey': os.getenv('GHOST_CONTENT_API_KEY')
}
g = ghost.GhostAdmin(**SITE_CONFIG)

REFRESH TOKEN


In [6]:
alle_posts = g.getAllPosts()

In [40]:
post = alle_posts[3]

body = ' '.join(BeautifulSoup(post['html']).stripped_strings)
body[:100]

'Hier op het blog was het stil, maar bij ons op school gebeurt er heel veel! Halverwege het vorige sc'

In [178]:

stemmer.stem("proef")

'proef'

# FullTextSearch in sqlite3 
https://www.sqlite.org/fts5.html

In [142]:
import sqlite3 
conn = sqlite3.connect(':memory:')
cur = conn.cursor()

`CREATE VIRTUAL TABLE ... USING FTS5(COLS, <OPTIONS>)`
 * `id unindexed` - ID wordt niet meegenomen in de search
 * `tokenize='trigram'` - experimenteel ding dat in delen van woorden zoekt, stemming is anders alleen Engels, en daar hebben we niks aan. 

In [143]:
cur.execute("""
  create virtual table posts using FTS5(id unindexed, title, body, tokens, tokenize='trigram')
""")
fields = 'id,title,body,tokens'.split(',')

In [145]:
# gebruik executemany om alles in 1 keer in te schieten, 
# en niet heel veel inserts (als backend dat ondersteund)
cur.execute('delete from posts;')
cur.executemany("""
  insert into posts values (?, ?, ?, ?)
""", [(post['uuid'], post['title'],' '.join(BeautifulSoup(post['html']).stripped_strings), stemmed(' '.join(BeautifulSoup(post['html']).stripped_strings))) for post in alle_posts])
cur.execute('commit')

In [179]:
body_idx = fields.index('body')
tokens_idx = fields.index('tokens')
needle = 'onderzoekjes proefjes leerlingen'
for row in cur.execute(f"""
    select bm25(posts), id, title, snippet(posts, {body_idx} ,'[',']','...',64), snippet(posts, {tokens_idx}, '[',']','...',64)
      from posts 
     where tokens match :stemmed
       and body match :original
     order by bm25(posts) desc 
     limit 10
    """,{'stemmed':stemmed(needle),'original':needle}):
    print(row[3])
    print(row[4])
    print()
    

...voorbeelden, [onderzoekjes] en [proefjes] voor [leerlingen], studies, cu...
... lesvoorbeeld [onderzoekjes] en [proefjes] voor [leerling] studies cursu...



In [182]:
body_idx = fields.index('body')
tokens_idx = fields.index('tokens')
needle = 'onderzoek proef leerling'
for row in cur.execute(f"""
    with stemmed as (
    select id
      from posts 
     where tokens match :stemmed
     order by bm25(posts) desc
    ) select posts.id, title, snippet(posts, {body_idx} ,'[',']','...',64)
      from posts inner join stemmed on posts.id = stemmed.id 
      where body match :original
      order by bm25(posts) desc
      limit 10
    """,{'stemmed':stemmed(needle),'original':needle}):
    print(row)
    print()
    

('6f446263-2bb7-461a-8296-acbefde5add0', 'Ready for the Ride - Moreel Leiderschap &amp; Moraal', '...‘De [leerling]en leren hoofdzaken over geestelijke stromingen die in...')

('da6ae360-47ab-4330-848a-9b32e7000851', 'Idee en LOF, een spannend stel…', '...svoorbeelden, [onderzoek]jes en [proef]jes voor [leerling]en, studies, c...')

('03f25d76-738d-4d99-9f2e-9de0ddf8974f', 'WIG wordt warm!', '...[caption id="attachment_7525" align="alignleft" width="404"] [Leerl]...')

('d6fc8fe6-17c4-4f28-8697-426dce8a600d', 'Ready for the Ride - Moreel Leiderschap &amp; Ouders', 'Voor het inkoppen Ouders - Het opvoeden van [leerling]en tot morele ...')

('071f9178-8d39-4011-8f48-1886ed884ddc', '4 je Talent', '...rbij [onderzoek]end leren centraal staat. [Leerling]en gaan [proef]onder...')

('762cff06-26a1-47d3-9265-b5cbfb0c3290', 'Inventiviteit bij natuurkunde', '...orden in [proef]jes en praktische opdrachten. Laat [leerling]en zelf u...')

